# Multiprocessing: limited direct execution; system calls
_COSC 208, Introduction to Computer Systems, 2023-04-12_

## Announcements
* DEI assignment 3 due Thursday @ 11pm
* Project 4 due Thursday, April 20

## Outline
* Operating Systems (OS)
* Accessing hardware
* Limited direct execution
* System calls

## Warm-up

* Q1: _Modify the `prediction` function to improve locality_

In [ ]:
char prediction(int *votes, char *projection) {
    int democrat = 0;
    int republican = 0;
    int length = strlen(projection);
    for (int d = 0; d < length; d++) {
        if (projection[d] == 'D') {
            democrat += votes[d];
        }
    }
    for (int r = 0; r < length; r++) {
        if (projection[r] == 'R') {
            republican += votes[r];
        }
    }
    if (democrat > republican) {
        return 'D';
    }
    else {
        return 'R';
    }
}

In [ ]:
// Perform loop fusion
char prediction(int *votes, char *projection) {
    int democrat = 0;
    int republican = 0;
    int length = strlen(projection);
    for (int v = 0; v < length; v++) {
        if (projection[v] == 'D') {
            democrat += votes[v];
        }
        if (projection[v] == 'R') {
            republican += votes[v];
        }
    }
    if (democrat > republican) {
        return 'D';
    }
    else {
        return 'R';
    }
}

<p style="height:4em;"></p>

* Q2: _Modify the `rowSum` function to improve locality_

In [ ]:
int *rowSum(int grid[][], int rows, int cols) {
    int *sums = malloc(sizeof(int) * rows);
    for (int i = 0; i < rows; i++) {
        sum[i] = 0;
    }
    for (int r = 0; r < rows; r++) {
        for (int c = 0; c < cols; c++) {
            sums[r] += grid[r][c];
        }
    }
    return sums;
}

In [ ]:
// Perform loop fusion
int *rowSum(int grid[][], int rows, int cols) {
    int *sums = malloc(sizeof(int) * rows);
    for (int r = 0; r < rows; r++) {
        sum[r] = 0;
        for (int c = 0; c < cols; c++) {
            sums[r] += grid[r][c];
        }
    }
    return sums;
}

<div style="page-break-after:always;"></div>

## Operating systems (OS) overview

* Purpose of an OS
    * Make computer hardware easy to use—e.g., an OS knows how to load an application's executable code from persistent storage (e.g., solid state drive (SSD)) into main memory, initialize the process's address space (code, heap, stack), and make the CPU execute the application's instructions
    * Support multiprocessing—i.e., running multiple applications concurrently
        * Concurrently == switch between multiple tasks during a window of time—e.g., alternate between cooking and setting the table
        * Simultaneously == complete two tasks at the same time—e.g., listen to a podcast while cooking
    * Allocate and manage hardware resources—e.g., decide when/which applications can use the CPU, decide when/which memory applications can use, prevent applications from stealing/accessing another application's CPU time or memory
    * Many OSes also provide a user interface (UI)
* How does the OS fulfill its duties?
    * Mechanisms — fundamental approaches for managing/providing access to hardware resources
        * E.g., system calls, process abstraction
    * Policy — specific ways of employing an approach; different policies make different trade-offs (in terms of efficiency, performance, etc.)
        * E.g., process scheduler

## Accessing hardware

* OS is responsible for allocating/managing the hardware
    * ⇒ applications should **not have unfettered access to hardware**
* _How should applications access the hardware?_
    * Ask the OS for access to the hardware
        * How do we ensure the OS does not "lose control" of the hardware? 
    * Asks the OS to perform an action on the application's behalf
        * How do we ensure this doesn't substantially degrade performance?
* Example: execute an instruction on the CPU
    * Asking the OS to do this on behalf of an application is impractical—OS would need to execute multiple assembly instructions for each assembly instruction the application wants to execute
    * Alternative: allow the application to execute its own instructions on the CPU
        * This is risky—an application may execute an instruction that controls the hardware, e.g., `hlt` (halt) instruction pauses the CPU
    * Alternative: allow the application to execute "safe" instructions on its own on the CPU
* Example: accessing the solid state drive (SSD)
    * Allowing the application to access the SSD directly
        * This is risky—an application may read/write data it should not be able to access
    * Alternative: asking the OS to access the SSD on the application's behalf
        * Latency of accessing SSD (~1 million CPU cycles) far outweighs the extra instructions required for OS to perform the access on the application's behalf
* Mechanisms
    * Limited Direct Execution (LDE)
    * System calls

## Limited Direct Execution

* CPU has two modes of execution: user mode & kernel mode
* _When does a CPU run in user mode?_ — when executing application code
* _When does a CPU run in kernel mode?_ — when executing OS code
* Allowable operations in user mode are restricted
    * Applications can...
        * Perform arithmetic/logic operations
        * Load/store values in its stack/heap
    * Applications must ask the OS to...
        * Start/terminate applications
        * Create/delete files
        * Display output on screen
        * Read input from user
    * Must transfer control to the OS to perform these operations — How?

## System calls

* Invoked via a special assembly instruction: trap (generic) or `svc` (on ARM)

* Example program

In [1]:
#include <stdio.h>
#include <unistd.h>
int user() {
    int uid = getuid();
    return uid;
}
int main() {
    int u = user();
    printf("User %d is running this process\n", u);
}

User 501 is running this process


* Assembly code

```
00000000004006ac <user>:
    4006ac:    d10083ff     sub    sp, sp, #0x20
    4006b0:    f9000bfe     str    x30, [sp, #16]
    4006b4:    94007713     bl    41e300 <__getuid>
    4006b8:    b9000fe0     str    w0, [sp, #12]
    4006bc:    b9400fe0     ldr    w0, [sp, #12]
    4006c0:    f9400bfe     ldr    x30, [sp, #16]
    4006c4:    910083ff     add    sp, sp, #0x20
    4006c8:    d65f03c0     ret
000000000041e300 <__getuid>:
    41e300:    d28015c8     mov    x8, #0xae
    41e304:    d4000001     svc    #0x0
    41e308:    d65f03c0     ret
```

* Functions in the C standard library that involve a privileged operation (e.g., `printf`) put the system call number in a register and invoke a trap instruction — programmer doesn’t have to worry about these details; they can just call the appropriate function in the C standard library
* When `svc` is executed
    1. CPU saves registers to the kernel stack — kernel stack is at a fixed location in memory
        * _Why do we need to save the registers?_ — so we can return to `user` when `__getuid` is done
    2. CPU switches to kernel mode
    3. CPU uses system call number to index into table of trap handlers 
        * At boot, initialize table of trap handlers with pointers into OS code for handling each type of syscall
    4. Branch and link to trap handler code
    5. CPU restores registers from the kernel stack
    6. CPU switches to user mode
    7. Resume execution after `svc` 
* _What should we do if an application tries to perform a privileged operation without making a system call?_
    * Let the code keep running — code may assume privileged operation was successful
    * Terminate the application